<a href="https://colab.research.google.com/github/seismosmsr/hawaii_soils/blob/main/Hawaii_Soils_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('sf')
install.packages('raster')
install.packages('exactextractr')
install.packages("googledrive")
install.packages("googletoken")
install.packages("DBI")
install.packages("RSQLite")
install.packages("RPostgreSQL")
install.packages("terra")
install.packages('randomForest')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘wk’, ‘classInt’, ‘Rcpp’, ‘s2’, ‘units’




In [ ]:
library(ggplot2)
library(sf)
library(magrittr)
library(data.table)
library(raster)
library(exactextractr)
library(googledrive)
library(DBI)
library(terra)
library(randomForest)
library(parallel)

In [ ]:
drive_auth(use_oob = TRUE, cache = FALSE)

In [ ]:
# https://drive.google.com/file/d/1ryTzWz0t9mgI5ynPvMzM21D135dLJvWv/view?usp=drive_link
# Updated: https://drive.google.com/file/d/17c3DJcujQ_0MHirG2z2lSU88A0lvjo1C/view?usp=sharing
drive_download(as_id('17c3DJcujQ_0MHirG2z2lSU88A0lvjo1C'), path = paste0('/content/fia.csv'), overwrite = TRUE)
fia_dt <-  st_read('/content/fia.csv') %>% st_as_sf(coords = c("LON","LAT"), crs = 4326) %>% data.table

In [ ]:
# https://drive.google.com/file/d/1G-qBpO_nzLIHWZQGNyrSutmBonjsrvEJ/view?usp=drive_link
drive_download(as_id('1G-qBpO_nzLIHWZQGNyrSutmBonjsrvEJ'), path = "/content/250_summary_grid_dt.gpkg", overwrite = TRUE)
grid_dt <- st_read("/content/250_summary_grid_dt.gpkg") %>% st_transform(4326) %>% data.table()

In [ ]:
# Simulation data
# https://drive.google.com/file/d/1HsEZDWJN7XO4aOZlMwbr_6MN3_-xUV81/view?usp=drive_link
drive_download(file = as_id('1HsEZDWJN7XO4aOZlMwbr_6MN3_-xUV81'), path =  '/content/sim_results_Hawaii_Run6.gpkg', overwrite = TRUE)
sim_dt <- st_read( '/content/sim_results_Hawaii_Run6.gpkg') %>% data.table

In [ ]:
grid_int_dt  <- fia_dt$geom %>% st_transform(4135) %>% st_buffer(800) %>% st_transform(4326) %>% st_intersects(grid_dt$geom)

In [ ]:
rm_vec <- grid_int_dt %>% lapply(length) %>% unlist > 0
grid_int_dt <- grid_int_dt[rm_vec]
fia_dt <- fia_dt[rm_vec,]

In [ ]:
grid_dt <- grid_dt[!is.na(srad),]

In [ ]:
grid_dt$agbd_n <- grid_dt$agbd_n %>% as.numeric

In [ ]:
fia_dt$Stock_SOC_Mg_Ha <- fia_dt$Stock_SOC_Mg_Ha %>% as.numeric

In [ ]:
sim_dt %>% names

In [ ]:
sim_dt[,z:=(predicted-Stock_SOC_Mg_Ha)/Stock_SOC_Mg_Ha]

In [ ]:
sim_dt[,residual_per:=(predicted-Stock_SOC_Mg_Ha)/Stock_SOC_Mg_Ha]

In [ ]:
sim_dt[id==2831164]$Stock_SOC_Mg_Ha %>% qplot(bins=100)

sim_dt[id==2831164 & z > -0.05 &z<0.05]$predicted %>% qplot(geom='density')

In [ ]:
sim_summary_dt <- sim_dt[,list(length = length(r2),length_05 = length(r2[sqrt((z)^2) <0.05]),geom=unique(geom)),by=.(id,PLOT)]

In [ ]:
# sim_summary_dt
sim_summary_dt[PLOT==5289]$length_05 %>% qplot

In [ ]:
sim_summary_dt %>% st_as_sf %>% st_write('sim_summary_id_plot.gpkg',delete_dsn = T)

In [ ]:
# sim_summary_dt

In [ ]:
sim_summary_dt %>%
  ggplot(aes(x= length_05/length)) +
  geom_histogram(bins=16)
    # geom_smooth(method = 'lm')

In [ ]:
sim_summary_dt[max(length_05/length)*.8 < length_05/length][order(length_05/length)]

In [ ]:
names(sim_dt)[39:68]

In [ ]:
sim_dt[,good:=abs(residual_per) < 0.1]

In [ ]:
# sim_dt$id %>% table

In [ ]:
names(sim_dt)[39:68] %>% lapply(function(x){
t.test(sim_dt[id ==2957662  & abs(residual_per) < 0.5 ][[x]],
sim_dt[id ==2957662  & abs(residual_per) >= 0.5 ][[x]])$p.value})


In [ ]:
(sim_dt %>% names)[40:69]

In [ ]:
 var_imp_dt<- sim_dt[ ][
  , .(p_value = lapply(.SD, function(x) {
    if(length(x[abs(z) < 0.05])>3 &length(x[abs(z) >= 0.05])>3 ){
      t.test(x[abs(z) < 0.05],
             x[abs(z) >= 0.05])$p.value
             }else(return(NA))

    }) ,
      variable = names(sim_dt)[39:69])
  , by = .(PLOT)
  , .SDcols = 39:69
]
var_imp_dt$p_value <- var_imp_dt$p_value %>% unlist

In [ ]:
# out_dt <- data.table(var_imp_dt,sim_summary_dt[match(var_imp_dt$id,sim_summary_dt$id)])
out_dt <- data.table(var_imp_dt,fia_dt[,c('geometry')][match(var_imp_dt$PLOT,fia_dt$PLOT)])

In [ ]:
out_dt[,bonferroni:=p_value*30]

In [ ]:
out_dt$p_value <- out_dt$p_value %>% unlist

In [ ]:
short_out_dt <- dcast(out_dt, PLOT  ~ variable,value.var='bonferroni', drop=FALSE)
short_out_dt <- data.table(short_out_dt,fia_dt[,c('geometry')][match(short_out_dt$PLOT,fia_dt$PLOT)])

In [ ]:
names(short_out_dt)

In [ ]:
cols_to_consider <- names(short_out_dt)[]


# Assuming 'short_out_dt' is your data.table
# Define the columns you want to consider for finding the max value by their indices
cols_to_consider_indices <- 3:(ncol(short_out_dt)-1) # Replace with actual indices of Var3, Var5, Var7
short_out_dt$max_col <-
1:nrow(short_out_dt) %>% lapply(function(x){
if(is.na(short_out_dt[x,cols_to_consider_indices,with=F] %>% unlist) %>% all){return(NA)}
short_out_dt[x,cols_to_consider_indices,with=F] %>% unlist %>% which.min( ) %>% names}) %>% unlist

In [ ]:
short_out_dt %>% st_write('max_col_p.gpkg')

In [ ]:
short_out_dt

In [ ]:
short_out_dt %>%
  ggplot(aes(x=1:nrow(short_out_dt),y=soil))+
  geom_line()

In [ ]:
out_dt[bonferroni <= .05]$variable %>% table

In [ ]:
out_dt[bonferroni<= 0.1] %>%
  ggplot(aes(x= variable %>% factor))+
  geom_bar()
  # geom_smooth(method='lm')

In [ ]:
data.table(out_dt) %>% st_as_sf() %>% st_write('/content/plot_sig_variables.gpkg',delete_dsn=T)

In [ ]:
# sim_dt[id ==2222069  ]
# 1850844
# sqrt(((Stock_SOC_Mg_Ha-predicted)/Stock_SOC_Mg_Ha)^2)<0.05
sim_dt[id ==2952969  & sqrt(((Stock_SOC_Mg_Ha-predicted)/Stock_SOC_Mg_Ha)^2)<0.05] %>%
  ggplot(aes(x=Stock_SOC_Mg_Ha)) +
  geom_histogram()
  # print()

In [ ]:
sim_dt %>%
  ggplot(aes(y=Stock_SOC_Mg_Ha,x=r2))+
    geom_bin2d()
    # geom_smooth()

In [ ]:
# Load the necessary library
library(repr)

# Adjust width and height of plots
options(repr.plot.width=20, repr.plot.height=8)